## Setting

In [125]:
import pandas as pd
import numpy as np
import datetime 
import seaborn as sns
import glob

In [126]:
df= pd.read_csv('master_final_220706.csv')
df= df.drop(columns="Unnamed: 0",axis=1)
df.head()

,cartype,예정시간,배차시간,승차시간,startpos1,startpos2,endpos1,endpos2,start_lat,start_lng,end_lat,end_lng,direction,distance,duration,taxiFare
0,중형승합,2018-01-23 00:04:00,2018-01-23 00:18:42,2018-01-23 00:34:08,성북구,장위제1동,동대문구,회기동,127.043713,37.614125,127.055313,37.590808,장위제1동회기동,4289,648.464,5600
1,중형승합,2018-01-23 00:17:00,2018-01-23 00:50:37,2018-01-23 01:19:22,중랑구,면목제7동,노원구,상계1동,127.086985,37.578995,127.054978,37.679901,면목제7동상계1동,14493,1318.675,13700
2,중형승합,2018-01-23 00:30:00,2018-01-23 00:17:06,2018-01-23 00:17:39,성북구,정릉제3동,강북구,수유제3동,127.004186,37.608958,127.023125,37.638715,정릉제3동수유제3동,4869,864.161,6200
3,중형승합,2018-01-23 00:47:00,2018-01-23 00:51:19,2018-01-23 01:09:33,구로구,고척제2동,관악구,은천동,126.858712,37.506706,126.942428,37.485309,고척제2동은천동,10247,1652.785,11000
4,중형승합,2018-01-23 01:08:00,2018-01-23 01:44:17,2018-01-23 02:06:53,동대문구,회기동,성북구,장위제2동,127.055313,37.590808,127.054521,37.612390,회기동장위제2동,3778,598.639,5200


In [127]:
df.shape

(407553, 16)

In [128]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 407553 entries, 0 to 407552
Data columns (total 16 columns):
 #   Column     Non-Null Count   Dtype  
---  ------     --------------   -----  
 0   cartype    407553 non-null  object 
 1   예정시간       407553 non-null  object 
 2   배차시간       407553 non-null  object 
 3   승차시간       407553 non-null  object 
 4   startpos1  407553 non-null  object 
 5   startpos2  407553 non-null  object 
 6   endpos1    407553 non-null  object 
 7   endpos2    407553 non-null  object 
 8   start_lat  407553 non-null  float64
 9   start_lng  407553 non-null  float64
 10  end_lat    407553 non-null  float64
 11  end_lng    407553 non-null  float64
 12  direction  407553 non-null  object 
 13  distance   407553 non-null  int64  
 14  duration   407553 non-null  float64
 15  taxiFare   407553 non-null  int64  
dtypes: float64(5), int64(2), object(9)
memory usage: 49.8+ MB


In [129]:
df[['distance','duration','taxiFare']].describe()

,distance,duration,taxiFare
count,407553.000000,407553.000000,407553.000000
mean,11227.803434,1427.078149,12017.347412
std,9384.646511,957.381900,8368.638228
min,205.000000,43.601000,0.000000
25%,4117.000000,679.163000,5730.000000
50%,8085.000000,1194.102000,9500.000000
75%,16139.000000,1918.202000,16300.000000
max,84449.000000,6880.172000,103750.000000


### 아이엠 콜택시 요금 산정
- 기본요금: 1000m(1km) 4000원 | 심야: 5000원 
- 거리요금: 120m(0.12km) 당 100원 
- 시간요금: 30초(0.5분) 당 100원 
- 기타요금: 시간거리와 완전 동시 병산▼
1) 심야할증 : 40% (0-4시)
2) 탄력요금제 : 0.8~4배 적용 -- 심야할증과 중복적용, 플랫폼 호출 고객 한정
3) 호출수수료 : 0~3000원

In [130]:
# 기본요금을 구합니다.

In [131]:
df["승차시간"]=pd.to_datetime(df["승차시간"])
df['승차시간_h'] = df['승차시간'].dt.hour
df['승차시간_h'].sort_values().unique()

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 23])

In [132]:
%%time 
기본_val = []

for i in range(len(df)):
    if df['승차시간_h'][i] in (0,1,2,3,4): ##심야시간 지정
        기본_cal = 5000
        기본_val.append(기본_cal) 
    else:
        기본_cal = 4000
        기본_val.append(기본_cal)
        
df['기본요금_아'] = 기본_val

CPU times: user 1.67 s, sys: 4.78 ms, total: 1.68 s
Wall time: 1.68 s


In [133]:
# 거리요금을 구합니다↓.--기본단위:m

In [134]:
%%time 
거리_val = []

for i in range(len(df)):
    if df['distance'][i]>1000:
        거리_cal = ((df['distance'][i]-1000)/120)*100
        거리_val.append(거리_cal)
    else:
        거리_val.append(0)
        
df['거리요금_아'] = 거리_val

CPU times: user 2.91 s, sys: 7.34 ms, total: 2.92 s
Wall time: 2.92 s


In [135]:
# 시간요금을 구합니다↓.--기본단위:초 

In [136]:
%%time 
시간_val = []

for i in range(len(df)):
    if df['duration'][i]>240: ##1Km 4분 제외한 추가 시간요금 구하기
        시간_cal = ((df['duration'][i]-240)/30)*100
        시간_val.append(시간_cal)
    else:
        시간_val.append(0)
        
df['시간요금_아'] = 시간_val

CPU times: user 2.92 s, sys: 6.28 ms, total: 2.92 s
Wall time: 2.92 s


In [138]:
df.iloc[:,-7:].head()

,distance,duration,taxiFare,승차시간_h,기본요금_아,거리요금_아,시간요금_아
0,4289,648.464,5600,0,5000,2740.833333,1361.546667
1,14493,1318.675,13700,1,5000,11244.166667,3595.583333
2,4869,864.161,6200,0,5000,3224.166667,2080.536667
3,10247,1652.785,11000,1,5000,7705.833333,4709.283333
4,3778,598.639,5200,2,5000,2315.000000,1195.463333


In [137]:
df[df['taxiFare']==0].iloc[:,-7:].head() ##거리가 1Km일떄 taxiFare=0-> 기본요금만 받는다.

,distance,duration,taxiFare,승차시간_h,기본요금_아,거리요금_아,시간요금_아
17,1000,240.0,0,3,5000,0.0,0.0
35,1000,240.0,0,5,4000,0.0,0.0
234,1000,240.0,0,7,4000,0.0,0.0
301,1000,240.0,0,7,4000,0.0,0.0
361,1000,240.0,0,7,4000,0.0,0.0


In [122]:
# 기타요금을 구합니다↓

In [123]:
from datetime import datetime,timedelta

In [124]:
#2) 탄력요금제는 수요에 따라 유동적으로 변환되는거라 정확한 기준 잡기 불가능. 탄력요금제 : 0.8~4배 적용 -- 심야할증과 중복적용, 플랫폼 호출 고객 한정
#3) 호출수수료 : 앱호출 경우 3000원

### 서울시 장애인 콜택시 요금 산정
- 기본요금: 5000m(5km) 1500원  
- 거리요금: 1000m(1km) 당 280원 | 10000m(10Km) 도달시 2900원 -> 10km 초과시: 1000m(1km) 당 70원
- 시간요금: x
- 기타요금: 지역간 할증x